# Wavelength calibration

In [5]:
import numpy as np
import glob
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'


### Loading crispy's dependencies

In [6]:
import sys
codefolder = '../../../../crispy'
if codefolder not in sys.path: sys.path.append(codefolder)
from crispy.tools.initLogger import getLogger
log = getLogger('crispy')
from crispy.params import Params
codefolder = '../../../crispy'
par = Params(codefolder)

### Generate monochromatic wavelengths

In [7]:
# Suppose we create a folder in crispy/ReferenceFiles/ called wavecalR50_770_test
par.wavecalDir = par.prefix+'/wavecalR50_770_test/'
# construct monochromatic wavelengths, doesn't really matter which wavelengths
lamlist = np.arange(700,845,10)
print(lamlist)


[700 710 720 730 740 750 760 770 780 790 800 810 820 830 840]


In [6]:
# construct images for each wavelength
from crispy.tools.wavecal import createWavecalFiles
createWavecalFiles(par,lamlist)

crispy - INFO - The number of input pixels per lenslet is 4.285714
crispy - INFO - Using PSFlet gaussian approximation
crispy - INFO - Final detector pixel per PSFLet: 40.000000
crispy - INFO - Processing wavelength 0.700000 (0 out of 1)
crispy - INFO - Number of detector pixels per lenslet: 13.384615
crispy - INFO - Rebinning final detector. Image has dimensions 1024x1024
crispy - INFO - Done.
crispy - INFO - Performance: 1 seconds total
crispy - INFO - Writing data to ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_700.fits
crispy - INFO - The number of input pixels per lenslet is 4.285714
crispy - INFO - Using PSFlet gaussian approximation
crispy - INFO - Final detector pixel per PSFLet: 40.000000
crispy - INFO - Processing wavelength 0.710000 (0 out of 1)


crispy - INFO - Number of detector pixels per lenslet: 13.384615
crispy - INFO - Rebinning final detector. Image has dimensions 1024x1024
crispy - INFO - Done.
crispy - INFO - Performance: 2 seconds total
crispy - INFO - Writing data to ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_710.fits
crispy - INFO - The number of input pixels per lenslet is 4.285714
crispy - INFO - Using PSFlet gaussian approximation
crispy - INFO - Final detector pixel per PSFLet: 40.000000
crispy - INFO - Processing wavelength 0.720000 (0 out of 1)
crispy - INFO - Number of detector pixels per lenslet: 13.384615
crispy - INFO - Rebinning final detector. Image has dimensions 1024x1024
crispy - INFO - Done.
crispy - INFO - Performance: 1 seconds total
crispy - INFO - Writing data to ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_720.fits
crispy - INFO - The number of input pixels per lenslet is 4.285714
crispy - INFO - Using PSFlet gaussian approximation
crispy - INFO - Final detector pixel per 

### Loading wavelengths/filelists

In [8]:
par.lamlist = np.loadtxt(par.wavecalDir + "lamsol.dat")[:, 0]
par.filelist = np.sort(glob.glob(par.wavecalDir + "det*.fits"))

### Run calibration. Choose parameters wisely.

In [9]:
from crispy.tools.wavecal import buildcalibrations
buildcalibrations(par,
                  order=3, # Order of polynomial map of the PSFLets (between 1 and 5)
                  upsample=10, # Upsampling factor of the PSFLets, needs to be at least ~5
                  nsubarr=1, # How many zones on the detector have their own PSFLets? 
                  genwavelengthsol=False, # Compute wavelength at the center of all pixels (need to do this at least once)
                  makehiresPSFlets=True, # Compute hires PSFLets (usually False)
                  parallel=True) # Use all computer cores

crispy - INFO - Building calibration files, placing results in ../../../crispy/ReferenceFiles/wavecalR50_770_test/
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_700.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_700.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_710.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_720.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_730.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_740.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_750.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_760.fits
crispy - INFO - Read data from HDU 1 of ../../../cris